<a href="https://colab.research.google.com/github/ykitaguchi77/Article_classifier/blob/main/Semantic_ScholarScholar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install ratelimit --q

  Preparing metadata (setup.py) ... done


In [50]:
# APIの設定

from google.colab import drive
drive.mount("/content/drive")

with open("/content/drive/MyDrive/Deep_learning/api.txt") as file:
    #text = file.read()
    i=1
    key = []
    while True:
        include_break_line = file.readline() #改行が含まれた行
        line = include_break_line.rstrip() #改行を取り除く
        if line: #keyの読み込み
            #print(f'{i}行目：{line}')
            key.append(line)
            i += 1
        else:
            break

# APIキーの準備
# #ngrok_aceess_token = key[5]
#openai_api_key = key[3]
# deepl_auth_key = key[1]
# serp_api_key = key[7]

import os
os.environ["OPENAI_API_KEY"] = key[3]
os.environ["SERPAPI_API_KEY"] = key[7]
os.environ["GOOGLE_CSE_ID"] = key[9]
os.environ["GOOGLE_API_KEY"] = key[11]
os.environ["PUBMED_API_KEY"] = key[27]
os.environ["SEMANTIC_SCHOLAR_API_KEY"] = key[29]

Mounted at /content/drive


In [51]:
SEMANTIC_API_KEY = os.environ["SEMANTIC_SCHOLAR_API_KEY"]  # APIキーを設定してください
result_limit = 10

ONE_MINUTE = 60

@sleep_and_retry
@limits(calls=1, period=ONE_MINUTE)  # /paper/search and /recommendations
def call_api_1_per_min(session, url, params):
    rsp = session.get(url, headers={'X-API-KEY': SEMANTIC_API_KEY}, params=params)
    rsp.raise_for_status()
    return rsp.json()

@sleep_and_retry
@limits(calls=10, period=ONE_MINUTE)  # other endpoints
def call_api_10_per_min(session, url, params):
    rsp = session.get(url, headers={'X-API-KEY': SEMANTIC_API_KEY}, params=params)
    rsp.raise_for_status()
    return rsp.json()

def requests_retry_session(
    retries=5,
    backoff_factor=0.3,
    status_forcelist=(429, 500, 502, 503, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def get_paper_id_by_title(title, session):
    rsp = call_api_1_per_min(session, 'https://api.semanticscholar.org/graph/v1/paper/search',
                            {'query': title, 'limit': 1, 'fields': 'paperId'})
    if rsp['total'] > 0:
        return rsp['data'][0]['paperId']
    else:
        return None

def get_paper_details(paper_id, session):
    return call_api_10_per_min(session, f'https://api.semanticscholar.org/graph/v1/paper/{paper_id}',
                            {'fields': 'title,abstract'})

def get_citations(paper_id, session):
    return _get_related_papers(session, f'https://api.semanticscholar.org/graph/v1/paper/{paper_id}/citations')

def get_references(paper_id, session):
    return _get_related_papers(session, f'https://api.semanticscholar.org/graph/v1/paper/{paper_id}/references')

def get_related_papers(paper_id, session):
    rsp = call_api_1_per_min(session, f"https://api.semanticscholar.org/recommendations/v1/papers/forpaper/{paper_id}",
                                {'fields': 'paperId', 'limit': 10})

    return [paper['paperId'] for paper in rsp['recommendedPapers']]


def _get_related_papers(session, url):
    edges = []
    params = {'fields': 'paperId', 'limit': 100}

    while True:
        try:
            rsp = call_api_10_per_min(session, url, params)
            print(f"Response for {url}: {json.dumps(rsp, indent=2)}")

            if 'data' in rsp and rsp['data']:
                for item in rsp['data']:
                    if 'citations' in url:
                        paper_key = 'citingPaper'
                    else:  # referencesの場合
                        paper_key = 'citedPaper'

                    if paper_key in item and 'paperId' in item[paper_key] and item[paper_key]['paperId']:
                        edges.append(item[paper_key]['paperId'])
            elif 'data' in rsp and not rsp['data']:
                print("No data received, exiting loop.")
                break

            if 'next' in rsp:
                params['offset'] = rsp['next']
                print(f"Next offset: {params['offset']}")
            else:
                print("No 'next' token, exiting loop.")
                break

        except requests.exceptions.RequestException as e:
            print(f"Error during API call: {e}")
            break  # エラー発生時はループを中断

    return edges


def get_paper_citation_and_reference_lists(paper_ids, session):
    paper_data = {}
    for paper_id in paper_ids:
        citations = get_citations(paper_id, session)
        time.sleep(1.2) # 1.2秒待つ
        references = get_references(paper_id, session)
        time.sleep(1.2) # 1.2秒待つ

        paper_data[paper_id] = {'citations': citations, 'references': references} # 全て終わったらpaper_dataに格納
    return paper_data

def cluster_papers(paper_details):
    if not paper_details:
        return {}

    texts = [f"{paper.get('title', '')} {paper.get('abstract', '')}" for paper in paper_details.values()]
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(texts)

    num_clusters = min(5, len(paper_details))
    kmeans = KMeans(n_clusters=num_clusters, random_state=0, n_init=10).fit(tfidf_matrix)

    clusters = {}
    for i, paper_id in enumerate(paper_details.keys()):
        clusters[paper_id] = kmeans.labels_[i]

    return clusters

def main():
    session = requests_retry_session()

    title = input('論文のタイトルを入力してください: ')
    paper_id = get_paper_id_by_title(title, session)

    if not paper_id:
        print("指定されたタイトルの論文が見つかりませんでした。")
        return

    print(f"論文ID: {paper_id}")

    citations = get_citations(paper_id, session)
    num_citations = len(citations) # citationの数を取得
    print(f"引用文献数: {num_citations}")

    references = get_references(paper_id, session)
    num_references = len(references) # referenceの数を取得
    print(f"被引用文献数: {num_references}")

    related_papers = get_related_papers(paper_id, session)
    num_related_papers = len(related_papers) # related_papersの数を取得
    print(f"関連論文数: {num_related_papers}")

    all_paper_ids = list(set(citations + references + related_papers + [paper_id]))
    print(f"全論文数（重複を除く）: {len(all_paper_ids)}") # 全体の数を表示

    paper_details = {}
    for pid in all_paper_ids:
        paper_details[pid] = get_paper_details(pid, session)

    citation_reference_lists = get_paper_citation_and_reference_lists(all_paper_ids, session)
    for pid, lists in citation_reference_lists.items():
        print(f"論文ID: {pid}, 引用文献数: {len(lists['citations'])}, 被引用文献数: {len(lists['references'])}")

    clusters = cluster_papers(paper_details)
    print("\nクラスタリング結果:")
    for paper_id, cluster in clusters.items():
        print(f"論文ID: {paper_id}, クラスタ: {cluster}")

if __name__ == '__main__':
    main()

論文のタイトルを入力してください: Blepharoptosis following ocular surgery: identifying risk factors
論文ID: f70b5e105a0f7d8be30141aa57d0e7996a2ccd05
Response for https://api.semanticscholar.org/graph/v1/paper/f70b5e105a0f7d8be30141aa57d0e7996a2ccd05/citations: {
  "offset": 0,
  "data": [
    {
      "citingPaper": {
        "paperId": "7322ce542a7d2406451844df173b4eb8908b1d70"
      }
    },
    {
      "citingPaper": {
        "paperId": "0fa0a826d36a000f3c9046ad9f0eb18883986ce6"
      }
    },
    {
      "citingPaper": {
        "paperId": "2497c06b0417897e02949f93e0adb14d41c284cc"
      }
    },
    {
      "citingPaper": {
        "paperId": "2860fff6283e2ab06ab8eaf2002fd7e3ad6b6231"
      }
    },
    {
      "citingPaper": {
        "paperId": "6cab326556588e17bd500fc640e19020c6c9b528"
      }
    },
    {
      "citingPaper": {
        "paperId": "0da4d1be0bfe334c446dc9f4267685fdda398968"
      }
    },
    {
      "citingPaper": {
        "paperId": "1528e10431fb4d31f986432c87eaced6f2c637ec"
 

KeyboardInterrupt: 